In [ ]:
import pandas as pd
import numpy as np
import statistics
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
from collections import OrderedDict


exchange_names=["ARCA","EDEX","NYSE"]
file_names=['arca_aapl.csv','edgex_aapl.csv','nyse_aapl.csv']
exchanges_df=[pd.read_csv(i) for i in file_names]
for i in exchanges_df:
    times= i['Time'].apply((lambda x: x.zfill(8)))
    i.index = times
    i = i.drop(columns=['Time'],inplace=True)


full_data=pd.concat(exchanges_df)
full_data.sort_index(inplace=True)
side_mapping = {
    1 : 'BID',
    2: 'OFFER',
}

full_data.Side = full_data.Side.map(side_mapping)
pd.set_option('display.max_columns', 10) 
pd.set_option('display.max_rows', 100000)  
pd.set_option('display.max_colwidth', -1)

In [ ]:
class Orderbook:
    
    def __init__(self):
        self.bid={}
        self.offer={}
        print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
        self.bid['ARCA']={'price':0, 'quantity':0 }
        self.bid['EDGEX']={'price':0, 'quantity':0 }
        self.bid['NYSE']={'price':0, 'quantity':0 }

        self.offer['ARCA']={'price':0, 'quantity':0 }
        self.offer['EDGEX']={'price':0, 'quantity':0 }
        self.offer['NYSE']={'price':0, 'quantity':0 }

    def process_tick(self,order):
        if order['side']=='BID':
            self.bid[order['exchange']].update({'price' : order['price'], 'quantity':order['quantity']})
        
        elif order['side']=="OFFER":
            self.offer[order['exchange']].update({'price' : order['price'], 'quantity' :order['quantity']})
           
        self.top_of_book()
        
    def top_of_book(self):
        self.od1 = OrderedDict(sorted(self.bid.items(), key=lambda x: x[1]['price'], reverse=True))
        self.od2 = OrderedDict(sorted(self.offer.items(), key=lambda x: x[1]['price'], reverse=False))
        print("BID",list(self.od1.items())[0]) 
        print("OFFER",list(self.od2.items())[0]) 
        print("************************************************************")
        
  #ANOTHER APPROACH      
#         max_price=self.bid['ARCA']['price']
#         quantity=self.bid['ARCA']['quantity']
#         exchang_max='ARCA'

#         for key,val in self.bid.items():
#             if val['price']>max_price:
#                 max_price=val['price']
#                 quantity=val['quantity']
#                 exchang_max=key
#         print("BID",max_price,quantity,exchang_max) 


#         min_price=self.offer['ARCA']['price']
#         quantity=self.offer['ARCA']['quantity']
#         exchang_min='ARCA'

#         for key,val in self.offer.items():
#             if val['price']<min_price:
#                 min_price=val['price']
#                 quantity=val['quantity']
#                 exchang_min=key
#         print("OFFER",min_price,quantity,exchang_min) 
#         print("************************************************************")
    
    def getBidVolumeBetween(self,price1, price2):
        vol=0
        for key,val in self.od1.items():
            if val['price']<=price2 and val['price']>=price1:
                vol+=val['quantity']
        return vol
    def getOfferVolumeBetween(self,price1, price2):
        vol=0
        for key,val in self.od2.items():
            if val['price']<=price2 and val['price']>=price1:
                vol+=val['quantity']
        return vol
        

In [ ]:
ob1 = Orderbook()
pprice=[i for i in full_data['Price']]
qquantity=[i for i in full_data['Volume']]
sside=[i for i in full_data['Side']]
eexchange=[i for i in full_data['Exchange']]
for i in range(0,len(pprice)):
    order={'price':pprice[i], 'quantity':qquantity[i], 'exchange' :eexchange[i], 'side' :sside[i]}
    ob1.process_tick(order)#uncomment print statements in class

In [ ]:
ob1.getBidVolumeBetween(100,120)